# 3 编码注意力机制

本章主要讲解流程如下：
1. 一个简单的自注意力
2. LLM中使用的注意力机制
3. 因果关系的注意力机制
4. 多头注意力机制

![1716780422961](../image/从零开始构建LLM/1716780422961.png)

## 3.1 长时序建模的问题

主要问题在于上下文丢失。如RNN不能在解码阶段直接从编码器中访问早期的隐藏状态。因此，它只依赖于当前的隐藏状态，它封装了所有相关的信息。这可能会导致上下文的丢失，特别是在依赖关系可能跨越较长距离的复杂句子中。


## 3.2 使用注意机制捕获数据依赖关系

早期为了解决RNN对于长时序问题，研究者提出以下结构，被成为*Bahdanau attention*，这一机制使得解码阶段能够访问编码早期状态。

![1718697183686](../image/从零开始构建LLM/1718697183686.png)

之后根据*Bahdanau attention*得到启发，提出了早期的*Transformer*结构。

![1716877433399](../image/从零开始构建LLM/1716877433399.png)

## 3.3 自注意输入的不同部分

自注意力是LLM中Transformer的基石。
在自注意力中，“自我”是指该机制通过关联单个输入序列中的不同位置来计算注意权重的能力。它关注的是本身不同部分的关系和依赖。而传统的注意力机制则是关注两个序列之间的关系。


### 3.3.1 一个简单的自我注意机制，没有训练权重

自注意的目标是为每个输入元素计算一个上下文向量，它结合了来自所有其他输入元素的信息。在自注意力中，我们的目标是为每一个输入元素${x^{(i)}}$计算上下文向量${z^{(i)}}$。一个上下文向量可以被解释为一个丰富的嵌入向量。</br>
如下图所示，*Your journey starts with one step*为输入句子，现在关注${x^{(2)}}$与${z^{(2)}}$，${z^{(2)}}$包含了从${x^{(1)}}$到${x^{(T)}}$之间的所有信息。
在自注意过程中，上下文向量起着至关重要的作用。它们的目的是通过在序列中合并来自所有其他元素的信息，在输入序列中（如句子）中创建每个元素的丰富表示，如下图所示。

![1716879045635](../image/从零开始构建LLM/1716879045635.png)

![1716881565809](../image/从零开始构建LLM/1716881565809.png)

In [1]:
import torch

In [2]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(f">> {attn_scores_2}")

>> tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


> 上述操作可以理解为矩阵的乘法 dot product，其中值越大，表示相关性越高

紧接着需要对其进行归一化操作

In [3]:
attn_scores_2 = attn_scores_2 / attn_scores_2.sum()
print(f">> attn_scores for x^2: {attn_scores_2}")
print(f">> attn_scores's sum for x^2: {attn_scores_2.sum()}")

>> attn_scores for x^2: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
>> attn_scores's sum for x^2: 1.0000001192092896


> 在实际中，更多的是使用softmax操作，这一操作在处理极值和梯度时有更好的表现。

In [4]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print(f">> attn_weights_naive for x^2: {attn_weights_2_naive}")
print(f">> attn_weights_naive's sum for x^2: {attn_weights_2_naive.sum()}")
print()

attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print(f">> attn_weights for x^2: {attn_weights_2}")
print(f">> attn_weights's sum for x^2: {attn_weights_2.sum()}")

>> attn_weights_naive for x^2: tensor([0.1630, 0.1770, 0.1765, 0.1603, 0.1570, 0.1663])
>> attn_weights_naive's sum for x^2: 1.0

>> attn_weights for x^2: tensor([0.1630, 0.1770, 0.1765, 0.1603, 0.1570, 0.1663])
>> attn_weights's sum for x^2: 1.0


In [5]:
# Above All
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print(f">> context_vec: {context_vec_2}")

>> context_vec: tensor([0.4325, 0.5937, 0.5349])


### 3.3.2 为所有输入计算权重

![1716945187106](../image/从零开始构建LLM/1716945187106.png)

计算流程与之前一致

![1716945198064](../image/从零开始构建LLM/1716945198064.png)

In [6]:
# >> attention scores
# method 1
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(f">> attn scores: {attn_scores}")

# method 2
attn_scores = torch.matmul(inputs, inputs.T)
print(f">> attn scores: {attn_scores}")

# >> attention weights (softmax)
attn_weights = torch.softmax(attn_scores, dim=1)
print(f">> attn weights (softmax): {attn_scores}")

>> attn scores: tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
>> attn scores: tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
>> attn weights (softmax): tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.

## 3.4 使用训练权重实现自注意力

### 3.4.1 一步一步计算注意力权重

这里引入了三个权重${W_q}$, ${W_k}$, ${W_v}$，这三个权重矩阵用于将输入token ${x^i}$ 映射为查询，键， 值向量。

![1716947702233](../image/从零开始构建LLM/1716947702233.png)

In [7]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

# requires_grad=False to reduce clutter in the outputs for illustration purposes
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

query_2 = torch.matmul(x_2, W_query)
key_2 = torch.matmul(x_2, W_key)
value_2 = torch.matmul(x_2, W_value)

attn_scores_22 = torch.dot(query_2, key_2)
print(attn_scores_22)

tensor(2.8946)


> 权重与注意力权重的区别：</br>
权重 ${W}$ 是指神经网络中的权重，在训练过程中被优化的部分。<br/>
注意权重决定了上下文向量依赖于输入的不同部分的程度。<br/>
<br/>
总的来说，权重参数是定义神经网络的基础的、可学习的参数，而注意力权重是上下文特定的、动态的值。


In [8]:
keys = torch.matmul(inputs, W_key)
values = torch.matmul(inputs, W_value)

print(f">> keys shape: {keys.shape}")
print(f">> values shape: {values.shape}")

attn_scores_2 = torch.matmul(query_2, keys.T)

d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1)
print(f">> attn weights for x_2: {attn_weights_2}")

context_vec_2 = torch.matmul(attn_weights_2, values)
print(f">> context vector for x_2: {context_vec_2}")

>> keys shape: torch.Size([6, 2])
>> values shape: torch.Size([6, 2])
>> attn weights for x_2: tensor([0.1197, 0.2658, 0.2574, 0.1116, 0.0817, 0.1638])
>> context vector for x_2: tensor([0.7908, 0.7233])


### 3.4.2 实现一个紧凑的自注意类

In [9]:
import torch.nn as nn


class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
  
    def forward(self, x):
        keys = torch.matmul(x, self.W_key)
        values = torch.matmul(x, self.W_value)
        queries = torch.matmul(x, self.W_query)

        attn_scores = torch.matmul(queries, keys.T)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        context_vec = torch.matmul(attn_weights, values)
        return context_vec

下图展示了注意力类的基本原理，声明$W_q$、$W_k$、$W_v$，维度与输入的序列对应（本质上应该与编码完成之后的对应上，即第2章中的output_dim），通过相乘得到Q、K、V三个矩阵，Q、K相乘再softmax得到注意力权重矩阵，注意力权重矩阵再与V相乘得到上下文向量。

![1717397936262](../image/从零开始构建LLM/1717397936262.png)

In [10]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(f">> context: {sa_v1(inputs)}")

>> context: tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


使用 `nn.Linear` ，除了可以有效计算矩阵外，它还优化了权值初始化方案，有助于模型训练更加稳定和有效

In [11]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
  
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = torch.matmul(queries, keys.T)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        context_vec = torch.matmul(attn_weights, values)
        return context_vec

In [12]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(f">> context: {sa_v2(inputs)}")

>> context: tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


## 3.5 用因果关系的注意力来隐藏未来的词语

其本质是通过构建一个上三角矩阵，将其后续部分掩盖，避免影响

![1717401849186](../image/从零开始构建LLM/1717401849186.png)

### 3.5.1 应用因果注意力掩码

![1717401901352](../image/从零开始构建LLM/1717401901352.png)

In [13]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)

attn_scores = torch.matmul(queries, keys.T)
attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
print(">> attn weights:", attn_weights)

context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(">> mask: ", mask_simple)

masked_simple = attn_weights * mask_simple
print(">> masked: ", masked_simple)

row_sums = torch.sum(masked_simple, dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(">> masked norm: ", masked_simple_norm)

>> attn weights: tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)
>> mask:  tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
>> masked:  tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBack

> **信息泄露**</br>
当应用掩码时，由于计算的权重已经进行了softmax，因此会有影响。然而，当我们在mask之后重新调整注意力权重时，本质是在一个更小的子集上重新计算softmax，因此mask位置对于softmax没有贡献。</br>

因此可以将流程简化为：

![1717402483387](../image/从零开始构建LLM/1717402483387.png)

In [14]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
print(">> mask: ", mask)

masked = torch.masked_fill(attn_scores, mask.bool(), -torch.inf)
print(">> masked: ", masked)

attn_weights = torch.softmax(masked / keys.shape[-1] ** 0.5, dim=1)
print(">> attn weights: ", attn_weights)

>> mask:  tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
>> masked:  tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)
>> attn weights:  tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<Sof

### 3.5.2 用dropout来掩盖额外的注意权重

在transformer架构中，dropout通常用在两个地方：**计算注意力分数之后**或者**应用注意力权重之前**

![1717558177482](../image/从零开始构建LLM/1717558177482.png)

需要注意的是，dropout时，会将原数值进行放大，这样能够保证注意力权重的平衡。

In [15]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
print(">> dropout rate (0.5): ", dropout(example))

print(">> dropout attn weights: ", dropout(attn_weights))

dropout = torch.nn.Dropout(0.1)
example = torch.ones(6, 6)
print(">> dropout rate (0.1): ", dropout(example))


>> dropout rate (0.5):  tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])
>> dropout attn weights:  tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.0000, 0.4638, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.3968, 0.3775, 0.3941, 0.0000],
        [0.3869, 0.3327, 0.0000, 0.0000, 0.3331, 0.3058]],
       grad_fn=<MulBackward0>)
>> dropout rate (0.1):  tensor([[1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111],
        [1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 0.0000],
        [0.0000, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111],
        [1.1111, 1.1111, 1.1111, 0.0000, 1.1111, 1.1111],
        [1.1111, 1.1111, 0.0000, 1.1111, 1.1111, 1.1111],
        [1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.

### 3.5.3 实现一个紧凑的因果注意力类

将上述的mask部分与dropout部分结合起来，因此现有的结构应当如下：

|内容|子项|
|---|---|
|结构|$W_q$, $W_k$, $W_v$|
||mask|
||dropout|
|||
|计算|$W_q$, $W_k$, $W_v$与输入相乘得到q, k, v|
||计算注意力分数并添加掩码|
||归一化得到权重|
||权重与v相乘得到上下文向量|

In [16]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = torch.matmul(queries, keys.transpose(1, 2))
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = torch.matmul(attn_weights, values)
        return context_vec

In [17]:
batch = torch.stack((inputs, inputs), dim=0)
print(">> batch shape: ", batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(">> context_vecs: ", context_vecs)
print(">> context_vecs.shape:", context_vecs.shape)

>> batch shape:  torch.Size([2, 6, 3])
>> context_vecs:  tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
>> context_vecs.shape: torch.Size([2, 6, 2])


## 3.6 将单个注意力扩展到多头注意力

### 3.6.1 将注意力扩展到多头注意力

这一部分将注意力扩展到多个，简单叠加即可

![1717567513041](../image/从零开始构建LLM/1717567513041.png)

In [18]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [19]:
torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(">> context_vecs:", context_vecs)
print(">> context_vecs.shape:", context_vecs.shape)

>> context_vecs: tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
>> context_vecs.shape: torch.Size([2, 6, 4])


### 3.6.2 通过权重分割实现多头注意力

将权重$W$拆分计算，最终合并，结果是一样的

![1718186662363](../image/从零开始构建LLM/1718186662363.png)

In [20]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False) -> None:
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = torch.matmul(queries, keys.transpose(2, 3))

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        context_vec = torch.matmul(attn_weights, values).transpose(1, 2)

        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vec

In [21]:
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(">> context_vecs:", context_vecs)
print(">> context_vecs.shape:", context_vecs.shape)

>> context_vecs: tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
>> context_vecs.shape: torch.Size([2, 6, 2])
